# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
# from azureml.core.dataset import Dataset
from azureml.core import Dataset, Datastore
from azureml.pipeline.steps import AutoMLStep

## Dataset

### Overview
Stellar Classification uses the spectral data of stars to categorize them into different categories. The modern stellar classification system is known as the Morgan–Keenan (MK) classification system. It uses the old HR classification system to categorize stars with their chromaticity and uses Roman numerals to categorize the star’s size.
In this Dataset, we will be using Absolute Magnitude and B-V Color Index to Identify Giants and Dwarfs.

#### Data content:
This Dataset contains several features of Stars:
1. Vmag: Visual Apparent Magnitude of the Star
2. Plx: Distance Between the Star and the Earth
3. e_Plx: Standard Error of Plx (Drop the Row if you find the e_Plx is too high!)
4. B-V: B-V color index. (A hot star has a B-V color index close to 0 or negative, while a cool star has a B-V color index close to 2.0.
5. Amag: Absolute Magnitude of the Star

#### Activity of Project:
In this capstone project we will be predicting the TargetClass given the above feature as input.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'demo-experiment'
experiment=Experiment(ws, experiment_name)

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "spectral-dataset"
description_text = "spectral data of stars dataset for udacity capstone project"
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/kalimi03/nd00333-capstone/master/Star3642_balanced.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe().dropna()
df.pop("SpType")
df.describe()

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass
count,3642.000000,3642.000000,3642.000000,3642.000000,3642.000000,3642.000000
mean,8.120530,7.606153,1.176568,0.827077,16.427388,0.500000
std,1.338875,11.924052,0.919999,0.436699,2.428144,0.500069
min,0.450000,-6.820000,0.470000,-0.196000,2.820000,0.000000
25%,7.370000,2.740000,0.840000,0.487250,15.150312,0.000000
50%,8.390000,4.950000,1.040000,0.867000,16.392485,0.500000
75%,9.060000,8.670000,1.270000,1.136000,18.029836,1.000000
max,12.490000,280.270000,30.290000,2.530000,29.249165,1.000000


## Creating or attaching a compute cluster target

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "demo-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

The details of Azure ML configuration settings have been explained in the ReadMe document.

In [7]:
project_folder = './insurance-project'
automl_settings = {
    "n_cross_validations": 5,
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "enable_early_stopping": True,
    "primary_metric": 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                            task = 'classification',
                            training_data=dataset,
                            label_column_name ="TargetClass",
                            path = project_folder,
                            **automl_settings
                            )

In [8]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on demo-cluster with default configuration
Running on remote compute: demo-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_f2ffd54f-172e-4409-bf3f-943e7cbaa3ed,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
# return details of the run
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_f2ffd54f-172e-4409-bf3f-943e7cbaa3ed,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_f2ffd54f-172e-4409-bf3f-943e7cbaa3ed',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-11T09:02:37.020886Z',
 'endTimeUtc': '2021-12-11T09:17:33.501673Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'demo-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"demo-experiment","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-165546","workspace_name":"quick-starts-ws-165546","region":"southcentralus","compute_target":"demo-cluster

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, model = remote_run.get_output()
best_run

Package:azureml-automl-runtime, training version:1.36.1, current version:1.34.0
Package:azureml-core, training version:1.36.0.post2, current version:1.34.0
Package:azureml-dataprep, training version:2.24.4, current version:2.22.2
Package:azureml-dataprep-rslex, training version:2.0.3, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.36.0, current version:1.34.0
Package:azureml-defaults, training version:1.36.0, current version:1.34.0
Package:azureml-inference-server-http, training version:0.4.2, current version:0.3.1
Package:azureml-interpret, training version:1.36.0, current version:1.34.0
Package:azureml-mlflow, training version:1.36.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.36.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.36.0, current version:1.34.0
Package:azureml-telemetry, training version:1.36.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.36.0, current v

Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_f2ffd54f-172e-4409-bf3f-943e7cbaa3ed_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0.3125, reg_lambda=2.3958333333333335, subsample=1, tree_method='auto'))], verbose=False)), ('15', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features='sqrt', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.01, min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1, oob_score=True, random_state=None, ver

In [13]:
model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(
      estimators=[('16', Pipeline(
          memory=None,
          steps=[('standardscalerwrapper', StandardScalerWrapper(
              copy=True,
              with_mean=False,
              with_std=False
          )), ('logisticregression', LogisticRegression(
              C=51.79474679231202,
              class_weight=None,
              dual=False,
              fit_intercept=True,
              intercept_scaling=1,
              l1_ratio=None,
              max_iter=100,
              multi_class='ovr',
              n_jobs=1,
              penalty='l2',
              random_state=Non

In [14]:
dir_name = 'infer_dir'
if dir_name not in os.listdir():
    os.mkdir(dir_name)

In [16]:
#TODO: Save the best model
import pickle
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'infer_dir/score.py')
best_run.download_file('outputs/model.pkl', 'infer_dir/model.pkl')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'infer_dir/conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
# Register the model
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model_name = best_run.properties['model_name']
run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace=ws,
                       model_name=model_name, # Name of the registered model in your workspace.
                       model_path='infer_dir/model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       description='Auto ML model to predict TargetClass.',
                       tags={'area': 'TargetClass', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model AutoMLf2ffd54f138
Name: AutoMLf2ffd54f138
Version: 1


In [18]:
# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="myenv", file_path="infer_dir/conda_env.yml")
inference_config = InferenceConfig(entry_script="infer_dir/score.py", environment=myenv)

# display the environment file
with open('infer_dir/conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.36.1
  - inference-schema
  - azureml-interpret==1.36.0
  - azureml-defaults==1.36.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
# Model Deployment
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "TargetClass", 'type': "classification"},
                                                           description="Predict TargetClass using classification model",
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service_name = "aci-medical-charges-deploy"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-11 09:26:21+00:00 Creating Container Registry if not exists..
2021-12-11 09:36:23+00:00 Registering the environment..
2021-12-11 09:36:25+00:00 Building image..
2021-12-11 09:47:55+00:00 Generating deployment configuration..
2021-12-11 09:47:56+00:00 Submitting deployment to compute.
2021-12-11 09:48:00+00:00 Checking the status of deployment aci-medical-charges-deploy..
2021-12-11 09:51:39+00:00 Checking the status of inference endpoint aci-medical-charges-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [21]:
# get the active api endpoint for scoring
print(aci_service.scoring_uri)

http://aa58c1f1-c1f3-4e59-9751-ec90590f34cb.southcentralus.azurecontainer.io/score


In [24]:
# run script to score the 4 observations below in the json payloader
import json
import requests

scoring_uri = 'http://aa58c1f1-c1f3-4e59-9751-ec90590f34cb.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

test_sample = json.dumps({
    "data": [
            [ 5.99, 13.73, 0.58, 1.318, 'K5III', 16.678352 ],
            [ 8.6, 5.09, 1.37, 0.448, 'B1II',  17.133589 ]
        ]
    })

response = requests.post(scoring_uri, data=test_sample, headers=headers)
print("Results:", response.json())

Results: {"result": [0, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
# print the log of the webservice
print(aci_service.get_logs())

2021-12-11T09:51:27,807740300+00:00 - iot-server/run 
2021-12-11T09:51:27,823879000+00:00 - rsyslog/run 
2021-12-11T09:51:27,806954100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-12-11T09:51:28,075146900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_bec455c772c2d018a0dc9c15319d853a/lib/libuuid.so.1: no version information available (required by rsyslogd)
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (69)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 89
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-12-11 09:51:33,020 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-12-11 09:51:33,023 | root | INFO | Starting up request id generator
2021-12-11 09:51:33,023 | root | INFO | Starting up app insight hooks
202

In [26]:
# delete the web service
aci_service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
